In [1]:
import ar1_approx as ar1
import numpy as np
import firm
from scipy import stats
import numba
import firm_exte_fina as firm2
import scipy

In [2]:
alpha=0.699
delta=0.15
psi=0.1647
beta=0.95
w=0.7
h=6.616
r=1/beta-1
sigma_z=0.857
rho=0.111
phi=0.1
theta1=[alpha,psi,rho,sigma_z]
theta2=[alpha,psi,rho,sigma_z,phi]
num_sample=1000
z_grid_size=9
num_sigma = 4

mu=0
num_draws=10000
moment_d1=np.array([0.03,0.24,0.4,0.25,3.0])
moment_d2=np.array([0.03,0.24,0.4,0.25,3.0,0.25])
u = np.random.uniform(size=[num_sample,num_draws])

In [3]:
@numba.jit
def sim_markov(z_grid, pi, num_draws,num_sample,u):
    # draw some random numbers on [0, 1]
    

    # Do simulations
    z_discrete = np.empty([num_sample,num_draws])  # this will be a vector of values 
    # we land on in the discretized grid for z
    N = z_grid.shape[0]
    oldind = np.random.randint(0,N,size=num_sample) # set initial value to median of grid
    z_discrete[:,0] = z_grid[oldind]
    z_ind=np.zeros([num_sample,num_draws])
    z_ind[:,0]=oldind
    for j in range(num_sample):
        oi=oldind[j]
        for i in range(1, num_draws):
            sum_p = 0
            ind = 0
            while sum_p < u[j,i]:
                sum_p = sum_p + pi[ind, oi]
#             print('inds =  ', ind, oldind)
                ind += 1
            if ind > 0:
                ind -= 1
            z_discrete[j,i] = z_grid[ind]
            z_ind[j,i]=ind
            oi = ind
                            
    return z_discrete,z_ind

In [4]:

z_grid_R=np.ones(z_grid_size)
step = (num_sigma * sigma_z) / (z_grid_size / 2)
pi_R, z_grid = ar1.rouwen(rho, mu, step, z_grid_size)
z_discrete_R,z_ind= sim_markov(z_grid_R, pi_R, num_draws,num_sample,u)
z_grid_R=np.exp(z_grid)

In [5]:
def data_moments(theta,*params):
    theta[0]=alpha
    theta[1]=psi
    theta[2]=rho
    theta[3]=sigma_z
    r,delta,num_sample,num_draws,u=params
    z_grid_size=9
    mu=0
    z_grid_R=np.ones(z_grid_size)
    step = (num_sigma * sigma_z) / (z_grid_size / 2)
    pi_R, z_grid = ar1.rouwen(rho, mu, step, z_grid_size)
    z_discrete_R,z_ind= sim_markov(z_grid_R, pi_R, num_draws,num_sample,u)
    z_grid_R=np.exp(z_grid)
    #find firm solution
    F,e,V=firm.firm_sol(alpha=alpha,delta=delta,psi=psi,r=r,z=z_grid_R,pi=pi_R)
    k_grid=F['kvec']
    k_grid_size=k_grid.shape[0]
    PF=F['PF']
    #create matrix containing index of k's, and generate initial k 
    k_ind=np.zeros([num_sample,num_draws])
    k_ind[:,0]=np.random.randint(0,k_grid_size,size=num_sample)
    #convert indice for k and z to integers for latter computation
    zi=[int(a) for a in np.nditer(z_ind)]
    z_index=np.matrix(zi).reshape(num_sample,num_draws)
    ki=[int(a) for a in np.nditer(k_ind)]
    k_index=np.matrix(ki).reshape(num_sample,num_draws)#only z_index and k_index are used
    
    #create matrice containg samples
    k_sample=np.zeros([num_sample,num_draws])
    k_sample[:,0]=list(k_grid[k_index[:,0]])
    v_sample=np.zeros([num_sample,num_draws])
    v_sample[:,0]=[V[i] for i in list(zip(np.array(z_index[:,0]),np.array(k_index[:,0])))]
    z_sample=[z_grid_R[i] for i in z_index]
    z_sample=np.matrix(np.array(z_sample)).reshape(1000,10000)
    #filling sample space for k,v
    p=1
    while p<num_draws:
        a=[PF[i] for i in list(zip(np.array(z_index[:,p-1]),np.array(k_index[:,p-1])))]#find position of optimal k 
        # from policy function
        b=[k_grid[i] for i in a] 
        k_sample[:,p]=b#put k' at each period into matrix for k
        k_index[:,p]=a #record indice for k' used as k at next period
        c=[V[i] for i in list(zip(np.array(z_index[:,p-1]),np.array(k_index[:,p-1])))]
        v_sample[:,p-1]=c #getting optimal firm value
        p+=1
    # data moments    
    I=k_sample[:,1:10000]-(1-delta)*k_sample[:,0:9999]
    op=np.multiply(z_sample,k_sample**alpha)
    I_K=I/k_sample[:,0:9999]
    q=v_sample/k_sample
    op_k=op/k_sample
    #calculate serial correlation
    sc_sample=np.ones(I_K.shape[0])
    for i in range(I_K.shape[0]):
        sc_sample[i]=stats.pearsonr(I_K[i,9498:9998],I_K[i,9499:9999])[0]
    sc=sc_sample.sum()/len(sc_sample)
    #calculate regression coefficients
    beta=np.zeros([500,3])
    for i in range(I_K.shape[0]):
        x=np.transpose(np.row_stack((np.ones(500),q[i,9498:9998],op_k[i,9499:9999])))
        y=I_K[i,9498:9998]
        beta[i:]=np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(x),x)),np.transpose(x)),y)
        B=beta.sum(axis=0)/beta.shape[0]
    
    #standard deviation for profit/k    
    std=np.sqrt(np.var(op_k[:,9498:9998],axis=1).sum())/num_sample
    average_q=v_sample[:,9498:9998].sum()/k_sample[:,9498:9998].sum()
    return B[1],B[2],sc, std,average_q

In [6]:
def data_moments2(theta,*params):
    theta[0]=alpha
    theta[1]=psi
    theta[2]=rho
    theta[3]=sigma_z
    theta[4]=phi
    r,delta,num_sample,num_draws,u=params
    z_grid_size=9
    mu=0
    z_grid_R=np.ones(z_grid_size)
    step = (num_sigma * sigma_z) / (z_grid_size / 2)
    pi_R, z_grid = ar1.rouwen(rho, mu, step, z_grid_size)
    z_discrete_R,z_ind= sim_markov(z_grid_R, pi_R, num_draws,num_sample,u)
    z_grid_R=np.exp(z_grid)
    #find firm solution
    F,e,V,exter=firm2.firm_sol(alpha=alpha,delta=delta,psi=psi,r=r,z=z_grid_R,pi=pi_R,phi=phi)
    k_grid=F['kvec']
    k_grid_size=k_grid.shape[0]
    PF=F['PF']
    #create matrix containing index of k's, and generate initial k 
    k_ind=np.zeros([num_sample,num_draws])
    k_ind[:,0]=np.random.randint(0,k_grid_size,size=num_sample)
    #convert indice for k and z to integers for latter computation
    zi=[int(a) for a in np.nditer(z_ind)]
    z_index=np.matrix(zi).reshape(num_sample,num_draws)
    ki=[int(a) for a in np.nditer(k_ind)]
    k_index=np.matrix(ki).reshape(num_sample,num_draws)#only z_index and k_index are used
    
    #create matrice containg samples
    k_sample=np.zeros([num_sample,num_draws])
    k_sample[:,0]=list(k_grid[k_index[:,0]])
    v_sample=np.zeros([num_sample,num_draws])
    v_sample[:,0]=[V[i] for i in list(zip(np.array(z_index[:,0]),np.array(k_index[:,0])))]
    z_sample=[z_grid_R[i] for i in z_index]
    z_sample=np.matrix(np.array(z_sample)).reshape(1000,10000)
    ex_fin_sample=np.zeros([num_sample,num_draws])
    #filling sample space for k,v
    p=1
    while p<num_draws:
        a=[PF[i] for i in list(zip(np.array(z_index[:,p-1]),np.array(k_index[:,p-1])))]#find position of optimal k 
        # from policy function
        b=[k_grid[i] for i in a] 
        k_sample[:,p]=b#put k' at each period into matrix for k
        k_index[:,p]=a #record indice for k' used as k at next period
        c=[V[i] for i in list(zip(np.array(z_index[:,p]),np.array(k_index[:,p])))]
        v_sample[:,p-1]=c #getting optimal firm value
        d=[exter[i] for i in list(zip(np.array(z_index[:,p]),np.array(k_index[:,p-1]),np.array(k_index[:,p])))]
        ex_fin_sample[:,p-1]=d
        p+=1
    
    # data moments    
    I=k_sample[:,1:10000]-(1-delta)*k_sample[:,0:9999]
    op=np.multiply(z_sample,k_sample**alpha)
    I_K=I/k_sample[:,0:9999]
    q=v_sample/k_sample
    op_k=op/k_sample
    #calculate serial correlation
    sc_sample=np.ones(I_K.shape[0])
    for i in range(I_K.shape[0]):
        sc_sample[i]=stats.pearsonr(I_K[i,9498:9998],I_K[i,9499:9999])[0]
    sc=sc_sample.sum()/len(sc_sample)
    #calculate regression coefficients
    beta=np.zeros([500,3])
    for i in range(I_K.shape[0]):
        x=np.transpose(np.row_stack((np.ones(500),q[i,9498:9998],op_k[i,9499:9999])))
        y=I_K[i,9498:9998]
        beta[i:]=np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(x),x)),np.transpose(x)),y)
        B=beta.sum(axis=0)/beta.shape[0]
    
    #standard deviation for profit/k    
    std=np.sqrt(np.var(op_k[:,9498:9998],axis=1).sum())/num_sample
    percentage_ex_fin=np.mean(ex_fin_sample[:,9498:9998].sum(axis=1)/500)
    average_q=v_sample[:,9498:9998].sum()/k_sample[:,9498:9998].sum()
    return B[1],B[2],sc, std,average_q,percentage_ex_fin

In [42]:
def moment_difference(theta,*params):
    moments_d=moment_d1
    moments_s=data_moments(theta,r,delta,num_sample,num_draws,u)
    moments_s=np.array(list(moments_s))
    difference=np.dot(np.dot(np.transpose(moments_d-moments_s),np.identity(moments_d.shape[0])),moments_d-moments_s)
    
    return difference

In [43]:
def moment_difference2(theta,*params):
    moments_d=moment_d2
    moments_s=data_moments2(theta,r,delta,num_sample,num_draws,u)
    moments_s=np.array(list(moments_s))
    difference=np.dot(np.dot(np.transpose(moments_d-moments_s),np.identity(moments_d2.shape[0])),moments_d-moments_s)
    
    return difference

In [ ]:
bound=[(-1, 1), (-1, 1), (-1,1),(-5,5)]
x=scipy.optimize.differential_evolution(moment_difference,bounds=bound,args=[moment_d1,r,delta,num_sample,num_draws,u])

In [ ]:
bound=[(-1, 1), (-1, 1), (-1,1),(-5,5),(-1,1)]
x=scipy.optimize.differential_evolution(moment_difference2,bounds=bound,args=[moment_d2,r,delta,num_sample,num_draws,u])

In [ ]:
moments_s1_prime=np.zeros([len(theta1),len(moments_s1)])
theta1_prime=np.row_stack([theta1,theta1,theta1,theta1])+np.identity(len(theta1))

for i in range(len(theta1)):

    moments_s1_prime[i,:]=np.array(list(data_moments(theta1_prime[i,:],r,delta,num_sample,num_draws,u)))
var1=np.dot(np.dot((moments_s1_prime-moments_s1),np.identity(len(moments_s1))),np.transpose(moments_s1_prime-moments_s1)).diagonal()

In [ ]:
moments_s2_prime=np.zeros([len(theta2),len(moments_s2)])
theta2_prime=np.row_stack([theta2,theta2,theta2,theta2,theta2])+np.identity(len(theta2))
for i in range(len(theta2)):

    moments_s2_prime[i,:]=np.array(list(data_moments2(theta2_prime[i,:],r,delta,num_sample,num_draws,u)))
    
var2=np.dot(np.dot((moments_s2_prime-moments_s2),np.identity(len(moments_s2))),np.transpose(moments_s2_prime-moments_s2)).diagonal()